Dataset
In these exercises, we’re going to experiment with the Chinook sample DB while using SQLAlchemy module

First, run the code below to download the database locally



### download and extract chinook sample DB
import urllib.request
import zipfile
from functools import partial
import os

chinook_url = 'http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip'
if not os.path.exists('chinook.zip'):
    print('downloading chinook.zip ', end='')
    with urllib.request.urlopen(chinook_url) as response:
        with open('chinook.zip', 'wb') as f:
            for data in iter(partial(response.read, 4*1024), b''):
                print('.', end='', flush=True)
                f.write(data)

zipfile.ZipFile('chinook.zip').extractall()
assert os.path.exists('chinook.db')


The helper methods below will help, you may use for the following exercises :



### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)



🌟 Exercise 1 : Open the database
open the database using sqlalchemy module interface. create an engine object in a variable named engine
call the connect() method to obtain a connection and place in a variable named cur
now run the code below to to run reflecton on the database, prepare classes that map to the database and create an orm session :

### useful: extract classes from the chinook database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [2]:
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)

In [3]:
### useful: extract classes from the chinook database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

NameError: name 'sqlalchemy' is not defined

In [9]:
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base
import pandas as pd

# Правильный путь к базе данных
db_path = '/content/chinook.db'

# Создаем движок SQLAlchemy для работы с локальной базой данных
engine = sqlalchemy.create_engine(f'sqlite:///{db_path}')

# Устанавливаем соединение с базой данных
connection = engine.connect()

# Отображаем (рефлектим) метаданные базы данных
metadata = sqlalchemy.MetaData()
metadata.reflect(bind=engine)

# Подготавливаем классы, отображающие таблицы базы данных
Base = automap_base(metadata=metadata)
Base.prepare()

# Создаем ORM-сессию
Session = sessionmaker(bind=engine)
session = Session()

# Функции для выполнения SQL-запросов и отображения результатов
def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)



In [10]:
table_names = metadata.tables.keys()

for table_name in table_names:
    print(table_name)

albums
artists
customers
employees
genres
invoice_items
tracks
media_types
invoices
playlist_track
playlists


In [12]:
Tracks = Base.classes.tracks

first_three_tracks = session.query(Tracks).limit(3).all()

print("Fitst three tracks:")
for track in first_three_tracks:
    print(f"ID: {track.TrackId}, Name: {track.Name}, Album ID: {track.AlbumId}, Length: {track.Milliseconds} мs")

Fitst three tracks:
ID: 1, Name: For Those About To Rock (We Salute You), Album ID: 1, Length: 343719 мс
ID: 2, Name: Balls to the Wall, Album ID: 2, Length: 342562 мс
ID: 3, Name: Fast As a Shark, Album ID: 3, Length: 230619 мс


In [14]:
Tracks = Base.classes.tracks
Albums = Base.classes.albums

query = session.query(Tracks.Name, Albums.Title).join(Albums, Tracks.AlbumId == Albums.AlbumId).limit(20)

print("FFirst 20 tracks and their albums:")
for track_name, album_title in query:
    print(f"Track: {track_name}, Album: {album_title}")

FFirst 20 tracks and their albums:
Track: For Those About To Rock (We Salute You), Album: For Those About To Rock We Salute You
Track: Put The Finger On You, Album: For Those About To Rock We Salute You
Track: Let's Get It Up, Album: For Those About To Rock We Salute You
Track: Inject The Venom, Album: For Those About To Rock We Salute You
Track: Snowballed, Album: For Those About To Rock We Salute You
Track: Evil Walks, Album: For Those About To Rock We Salute You
Track: C.O.D., Album: For Those About To Rock We Salute You
Track: Breaking The Rules, Album: For Those About To Rock We Salute You
Track: Night Of The Long Knives, Album: For Those About To Rock We Salute You
Track: Spellbound, Album: For Those About To Rock We Salute You
Track: Balls to the Wall, Album: Balls to the Wall
Track: Fast As a Shark, Album: Restless and Wild
Track: Restless and Wild, Album: Restless and Wild
Track: Princess of the Dawn, Album: Restless and Wild
Track: Go Down, Album: Let There Be Rock
Track: Dog

In [15]:
InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks

query = session.query(Tracks.Name, InvoiceItems.Quantity).join(Tracks, InvoiceItems.TrackId == Tracks.TrackId).limit(10)

print("First 10 sales:")
for track_name, quantity in query:
    print(f"Track: {track_name}, QTY Sold {quantity}")

First 10 sales:
Track: Balls to the Wall, QTY Sold 1
Track: Restless and Wild, QTY Sold 1
Track: Put The Finger On You, QTY Sold 1
Track: Inject The Venom, QTY Sold 1
Track: Evil Walks, QTY Sold 1
Track: Breaking The Rules, QTY Sold 1
Track: Dog Eat Dog, QTY Sold 1
Track: Overdose, QTY Sold 1
Track: Love In An Elevator, QTY Sold 1
Track: Janie's Got A Gun, QTY Sold 1


In [16]:
# Получаем классы для таблиц 'invoice_items' и 'tracks'
InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks

query = session.query(Tracks.Name, sqlalchemy.func.sum(InvoiceItems.Quantity).label('total_sold'))\
               .join(Tracks, InvoiceItems.TrackId == Tracks.TrackId)\
               .group_by(Tracks.Name)\
               .order_by(sqlalchemy.desc('total_sold'))\
               .limit(10)

print("Top 10 mnost selling tracks:")
for track_name, total_sold in query:
    print(f"Track: {track_name}, Times sold: {total_sold}")

Top 10 mnost selling tracks:
Track: The Trooper, Times sold: 5
Track: Untitled, Times sold: 4
Track: The Number Of The Beast, Times sold: 4
Track: Sure Know Something, Times sold: 4
Track: Hallowed Be Thy Name, Times sold: 4
Track: Eruption, Times sold: 4
Track: Where Eagles Dare, Times sold: 3
Track: Welcome Home (Sanitarium), Times sold: 3
Track: Sweetest Thing, Times sold: 3
Track: Surrender, Times sold: 3


In [17]:
InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks
Albums = Base.classes.albums
Artists = Base.classes.artists

query = session.query(Artists.Name, sqlalchemy.func.sum(InvoiceItems.Quantity).label('total_sold'))\
               .join(Tracks, InvoiceItems.TrackId == Tracks.TrackId)\
               .join(Albums, Tracks.AlbumId == Albums.AlbumId)\
               .join(Artists, Albums.ArtistId == Artists.ArtistId)\
               .group_by(Artists.Name)\
               .order_by(sqlalchemy.desc('total_sold'))\
               .limit(10)

# Выведем результат
print("Top 10 most selling artists:")
for artist_name, total_sold in query:
    print(f"Artist: {artist_name}, Tracks sold: {total_sold}")

Top 10 most selling artists:
Artist: Iron Maiden, Tracks sold: 140
Artist: U2, Tracks sold: 107
Artist: Metallica, Tracks sold: 91
Artist: Led Zeppelin, Tracks sold: 87
Artist: Os Paralamas Do Sucesso, Tracks sold: 45
Artist: Deep Purple, Tracks sold: 44
Artist: Faith No More, Tracks sold: 42
Artist: Lost, Tracks sold: 41
Artist: Eric Clapton, Tracks sold: 40
Artist: R.E.M., Tracks sold: 39
